In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation
import datetime
from datetime import timedelta as delta
from matplotlib import rc
from mpl_toolkits.basemap import Basemap

In [ ]:
class ParticleData(object):
    def __init__(self):
        self.id = []

def load_particles_file(fname, varnames):
    T = ParticleData()
    pfile = Dataset(fname, 'r')
    T.id = pfile.variables['trajectory'][:]

    for v in varnames:
        setattr(T, v, pfile.variables[v][:].T)
    T.time_origin = datetime.datetime(1900, 12, 31)
    
    T.lonends = np.zeros(T.lon.shape[0])
    T.latends = np.zeros(T.lon.shape[0])
    for i in range(T.lon.shape[0]-1):
        lon = T.lon[i,:]
        T.lonends[i] = lon[np.isfinite(lon)][-1]
        lat = T.lat[i,:]
        T.latends[i] = lat[np.isfinite(lat)][-1]
    return T

Tall = {}
for s in range(1, 13):
    Tall[s] = load_particles_file('antarcticplastic_1yr_%02d.nc' % s, ['lon', 'lat', 'time'])

In [ ]:
plt.rcParams['figure.figsize'] = (14, 7)
fig = plt.figure()
axlims = [-75, -50, -150+360, -30+360]
m = Basemap(projection='cyl', llcrnrlat=axlims[0], urcrnrlat=axlims[1], llcrnrlon=axlims[2], urcrnrlon=axlims[3], resolution='i')
m.drawcoastlines()
m.fillcontinents(color='burlywood')
m.drawparallels(range(-90, 0, 5), labels=[True,False,False,False])
m.drawmeridians(range(0, 360, 5), labels=[False,False,True,False])

for i, s in enumerate(Tall):
    xs, ys = m(Tall[s].lon, Tall[s].lat)
    m.plot(xs[0], ys[0], '.', color=plt.cm.hsv(float(i)/len(Tall)), ms=10, label='Sampling Point #%d'%s)
    m.plot(xs[1:], ys[1:], '.', color=plt.cm.hsv(float(i)/len(Tall)), ms=2)

plt.legend(loc=1)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (14, 20)
fig = plt.figure()
for i, s in enumerate(Tall):
    ax = fig.add_subplot(6, 2, i+1)
    m = Basemap(projection='cyl', llcrnrlat=axlims[0], urcrnrlat=axlims[1], llcrnrlon=axlims[2], urcrnrlon=axlims[3], resolution='i')
    m.drawcoastlines()
    m.fillcontinents(color='burlywood')
    m.drawparallels(range(-90, 0, 10), labels=[True,False,False,False])
    m.drawmeridians(range(0, 360, 10), labels=[False,False,False, True])
    ax.set_title('Sampling Point #%d'%s)

    xs, ys = m(Tall[s].lon, Tall[s].lat)
    m.plot(xs, ys, '.', color=plt.cm.hsv(float(i)/len(Tall)), ms=2)

plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (14, 5)
def makemov(Tall):
    fig = plt.figure()
    frames = np.arange(0, 365)
    
    def setmap():
        ax = fig.add_subplot(1, 1, 1)
        ttl = ax.set_title('Particles 0 days before sampling')
        m = Basemap(projection='cyl', llcrnrlat=axlims[0], urcrnrlat=axlims[1], llcrnrlon=axlims[2], urcrnrlon=axlims[3], resolution='i')
        m.drawcoastlines()
        m.drawparallels(range(-90, 0, 5), labels=[True,False,False,False])
        m.drawmeridians(range(0, 360, 5), labels=[False,False,False,True])
        m.fillcontinents(color='burlywood')
        return m, ttl

    def setscat(T, m, col):
        xs, ys = m(T.lon[frames[0], :], T.lat[frames[0], :])
        scat = m.scatter(xs, ys, c=col, s=40, edgecolor='k', label='Sampling Point #%d'%s)
        return scat

    m, ttl = setmap()
    scat = {}
    for i, s in enumerate(Tall):
        scat[i] = setscat(Tall[s], m, plt.cm.hsv(float(i)/len(Tall)))
    plt.legend(loc=1)
    
    def animate(t):
        for i, s in enumerate(Tall):
            if Tall[s].lon.shape[0] > t:
                xs, ys = m(Tall[s].lon[t, :], Tall[s].lat[t, :])
                scat[i].set_offsets(np.matrix((xs, ys)).transpose())
        ttl.set_text('Particles %d days before sampling' % t)
        return scat

    rc('animation', html='html5')
    anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100, blit=False)
    
    plt.close()
    return anim
makemov(Tall)